# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021)

In [4]:
import requests
import pandas as pd
import numpy as np

In [3]:
with open('data.txt', 'r') as file:
  data = file.read()
data[:100]

'141\n140\n160\n161\n162\n172\n178\n185\n184\n186\n187\n195\n216\n239\n243\n247\n248\n243\n224\n230\n227\n228\n229\n231\n220\n'

In [8]:
df = pd.DataFrame([int(d) for d in data.split('\n')], columns=['depth'])
df['diff'] = df.depth.diff()
df

,depth,diff
0,141,NaN
1,140,-1.0
2,160,20.0
3,161,1.0
4,162,1.0
...,...,...
1995,9081,1.0
1996,9084,3.0
1997,9089,5.0
1998,9102,13.0


In [10]:
df.query('diff > 0').shape

(1692, 2)

In [13]:
df.depth.astype('float', copy=False)

0        141.0
1        140.0
2        160.0
3        161.0
4        162.0
         ...  
1995    9081.0
1996    9084.0
1997    9089.0
1998    9102.0
1999    9105.0
Name: depth, Length: 2000, dtype: float64

In [18]:
df['window'] = df.depth.rolling(3).sum()
df['window_diff'] = df.window.diff()
df.query('window_diff > 0').shape

(1724, 4)